In [1]:
import pandas as pd
import numpy as np

In [2]:
tbd = pd.read_excel('tbd.xlsx')
tbd

,enumerator_name,location,gender,age_range,qual_life1,qual_life8,qual_life9,qual_life12,qual_life13,qual_life14,...,rapport3,partner_feedback,security1,security2,recognition,justice,recognition_justice,dignity,emergency_irm,comments
0,سعد برو,gaziantep,male,25-55,good,not_at_all,moderate_amount,completely,a_little,a_little,...,yes,تشكر الجهود فقط,no,NaN,very_much,very_much,very_much,very_much,نقلة المنزل,لا
1,سعد برو,gaziantep,male,25-55,good,not_at_all,moderate_amount,moderately,mostly,a_little,...,yes,كانو متعاونين إلى حد كبير,no,NaN,very_much,a_little,a_little,somewhat,صرف على أجور المعيشة والتنقل وقت الزلزال,يفضل ان يتم إعطاء الدعم المناسب لكل حالة حسب و...
2,سعد برو,gaziantep,male,25-55,neither,moderate_amount,very_much,not_at_all,mostly,not_at_all,...,yes,مسرور من المشاركين بالمشروع,no,NaN,totally,totally,very_much,very_much,اجار منزل في اسطنبول,لا
3,سعد برو,gaziantep,male,25-55,neither,not_at_all,very_much,a_little,moderately,a_little,...,yes,لا كل الشكر,no,NaN,very_much,very_much,totally,very_much,اجار للمنزل,يفضل ان يسرع المشروع بسبب الحاجة للدعم المادي
4,سعد برو,gaziantep,female,over_55,very_poor,not_at_all,a_little,a_little,a_little,moderately,...,yes,مشكورين,no,NaN,very_much,very_much,very_much,somewhat,ساعدت فيه ابنتي ودفعت اجار البيت والفواتير,اتمنى ان يتم اضافة المساعدات الطبية من حيث الت...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,Basil Haj Hussein,urfa,male,25-55,good,extreme_amount,a_little,moderately,a_little,completely,...,somewhat,NaN,no,NaN,somewhat,not_at_all,not_at_all,not_at_all,NaN,NaN
102,Basil Haj Hussein,urfa,male,25-55,poor,not_at_all,a_little,moderately,a_little,completely,...,yes,NaN,no,NaN,totally,totally,totally,totally,المستلزمات الأساسية خلال فترة الزلزال التي خرج...,اتمنى ان يكون المبلغ جيد بما يكفي احتياجي لإجر...
103,Basil Haj Hussein,urfa,male,over_55,neither,not_at_all,a_little,not_at_all,mostly,not_at_all,...,yes,NaN,yes,اتخوف من تسرب المعلومات الشخصية للمشاركين وذلك...,somewhat,a_little,totally,somewhat,NaN,انا كفيل للأيتام وأرجو مساعدتي بتأمين أجور تدر...
104,Basil Haj Hussein,urfa,female,25-55,neither,a_little,moderate_amount,a_little,a_little,moderately,...,yes,NaN,yes,لا أعلم,very_much,somewhat,very_much,very_much,NaN,NaN
